# Final Captone Project

In [106]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


## 1. the neighborhoods information at Huouston

In [116]:
# from this website

url = 'https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Houston'


In [133]:
df1 = pd.read_csv('houston.csv')
df1.head(100)

,Neighborhood
0,Willowbrook
1,Greater Greenspoint
2,Carverdale
3,Fairbanks
4,Greater Inwood
5,Acres Home
6,Hidden Valley
7,Westbranch
8,Addicks / Park Ten
9,Spring Branch


## 2. Get the geographical coordinates

In [134]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Houston, Texas'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [135]:
coords = [ get_latlng(neighborhood) for neighborhood in df1["Neighborhood"].tolist() ]

In [136]:
coords

[[29.95240000000007, -95.54462999999998],
 [29.939670000000035, -95.40747999999996],
 [29.849590000000035, -95.54244999999997],
 [29.852730000000065, -95.52418999999998],
 [29.869770000000074, -95.48043999999999],
 [29.87047000000007, -95.43535999999995],
 [29.88847000000004, -95.41459999999995],
 [29.83937000000003, -95.55360999999999],
 [29.814340000000072, -95.61592999999999],
 [29.803508801066044, -95.51587497352799],
 [29.826440000000048, -95.48876999999999],
 [29.847092588973737, -95.49484945813951],
 [29.82797000000005, -95.39369999999997],
 [29.801301000000024, -95.43099099999995],
 [29.796250000000043, -95.39486999999997],
 [29.77263000000005, -95.57091999999994],
 [29.674794475323573, -95.6231582925845],
 [29.74768000000006, -95.57423999999997],
 [29.725484461288932, -95.55103598948064],
 [29.732170000000053, -95.52336999999994],
 [29.760150000000067, -95.47670999999997],
 [29.888899459838154, -95.32449993087188],
 [29.803758824075437, -95.47214984877535],
 [29.76406700000006

In [137]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [138]:
df1['Latitude'] = df_coords['Latitude']
df1['Longitude'] = df_coords['Longitude']

In [139]:
# check the neighborhoods and the coordinates
print(df1.shape)
df1.head(100)

(88, 3)


,Neighborhood,Latitude,Longitude
0,Willowbrook,29.952400,-95.544630
1,Greater Greenspoint,29.939670,-95.407480
2,Carverdale,29.849590,-95.542450
3,Fairbanks,29.852730,-95.524190
4,Greater Inwood,29.869770,-95.480440
5,Acres Home,29.870470,-95.435360
6,Hidden Valley,29.888470,-95.414600
7,Westbranch,29.839370,-95.553610
8,Addicks / Park Ten,29.814340,-95.615930
9,Spring Branch,29.803509,-95.515875


## Use the Foursquare API to explore the neighborhoods

In [142]:
# define Foursquare Credentials and Version
CLIENT_ID = 'Z1FWBLDZ1SCVDHX4ZMPWHJOJ1RTKOKLI31WJ2EKSN20HMHLM' # your Foursquare ID
CLIENT_SECRET = 'BJPFB15Q2WE0GGINOWXZSJR5WXA1B3JCRPM1SHRW5SKVVYUK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: Z1FWBLDZ1SCVDHX4ZMPWHJOJ1RTKOKLI31WJ2EKSN20HMHLM
CLIENT_SECRET:BJPFB15Q2WE0GGINOWXZSJR5WXA1B3JCRPM1SHRW5SKVVYUK


In [150]:
df = df1

In [151]:
radius = 2000
LIMIT = 200

venues = []

for lat, long, neighborhood in zip(df['Latitude'], df['Longitude'], df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [152]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(5115, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Willowbrook,29.9524,-95.54463,Bed Bath & Beyond,29.953517,-95.543865,Furniture / Home Store
1,Willowbrook,29.9524,-95.54463,Babin's Seafood House,29.955088,-95.544452,Seafood Restaurant
2,Willowbrook,29.9524,-95.54463,Costco,29.954658,-95.547697,Warehouse Store
3,Willowbrook,29.9524,-95.54463,buybuy BABY,29.953127,-95.543557,Kids Store
4,Willowbrook,29.9524,-95.54463,Pho An 2,29.956606,-95.543805,Vietnamese Restaurant


In [153]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Acres Home,7,7,7,7,7,7
Addicks / Park Ten,5,5,5,5,5,5
Afton Oaks,100,100,100,100,100,100
Alief,64,64,64,64,64,64
Astrodome Area,47,47,47,47,47,47
Braeburn,57,57,57,57,57,57
Braeswood,83,83,83,83,83,83
Brays Oaks,58,58,58,58,58,58
Briar Forest,100,100,100,100,100,100


In [154]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 329 uniques categories.


In [452]:
test = venues_df.groupby(["VenueCategory"]).count()


In [455]:
test.sort_values(by='Neighborhood',ascending=False)

,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude
VenueCategory,,,,,,
Mexican Restaurant,257,257,257,257,257,257
Pizza Place,184,184,184,184,184,184
Fast Food Restaurant,179,179,179,179,179,179
Sandwich Place,154,154,154,154,154,154
Discount Store,145,145,145,145,145,145
Fried Chicken Joint,125,125,125,125,125,125
Coffee Shop,120,120,120,120,120,120
Gas Station,107,107,107,107,107,107
Burger Joint,96,96,96,96,96,96


In [456]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:10]

array(['Furniture / Home Store', 'Seafood Restaurant', 'Warehouse Store',
       'Kids Store', 'Vietnamese Restaurant', 'Steakhouse', 'Wings Joint',
       'Sporting Goods Shop', 'Thai Restaurant',
       'Health & Beauty Service'], dtype=object)

## Analyze Each Neighborhood

In [457]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()


(5115, 330)


,Neighborhoods,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Football Field,College Rec Center,College Residence Hall,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fireworks Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Marine Terminal,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Port,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rugby Pitch,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Sk

In [509]:
kl_grouped = kl_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(88, 330)


,Neighborhoods,ATM,Accessories Store,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bike Trail,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Casino,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Baseball Diamond,College Football Field,College Rec Center,College Residence Hall,College Theater,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Costume Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Financial or Legal Service,Fireworks Store,Flea Market,Floating Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gun Shop,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Locksmith,Lounge,Luggage Store,Mac & Cheese Joint,Marine Terminal,Market,Martial Arts School,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Mosque,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Port,Portuguese Restaurant,Post Office,Print Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rugby Pitch,Salad Place,Salon / Barbershop,Salvadoran Restaurant,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shopping Mall,Sk

## Check Restaurant

In [510]:
#kl_gym changed to kl_r
kl_r =kl_grouped[["Neighborhoods","Mexican Restaurant",'Pizza Place','Fast Food Restaurant','Sandwich Place','Fried Chicken Joint','Burger Joint', \
                    'American Restaurant','Vietnamese Restaurant','Chinese Restaurant','Seafood Restaurant','Italian Restaurant']]

In [511]:
kl_r.head()

,Neighborhoods,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Burger Joint,American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Seafood Restaurant,Italian Restaurant
0,Acres Home,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.142857,0.00
1,Addicks / Park Ten,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,Afton Oaks,0.050000,0.050000,0.050000,0.060000,0.020000,0.01,0.000000,0.020000,0.010000,0.040000,0.03
3,Alief,0.000000,0.015625,0.031250,0.015625,0.031250,0.00,0.000000,0.265625,0.000000,0.000000,0.00
4,Astrodome Area,0.021277,0.021277,0.021277,0.063830,0.021277,0.00,0.021277,0.000000,0.021277,0.000000,0.00


## Cluster Neighborhoods

In [512]:
# set number of clusters
kclusters = 5

#original
#kl_clustering = kl_gym.drop(["Neighborhoods"], 1)

#change to:
kl_clustering = kl_r.drop(["Neighborhoods"], 1)

print(kl_clustering.shape)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

(88, 11)


array([3, 0, 2, 4, 0, 0, 0, 1, 0, 0])

In [513]:

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_r.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [514]:

kl_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
kl_merged.head()

,Neighborhood,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Burger Joint,American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Cluster Labels
0,Acres Home,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.142857,0.00,3
1,Addicks / Park Ten,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0
2,Afton Oaks,0.050000,0.050000,0.050000,0.060000,0.020000,0.01,0.000000,0.020000,0.010000,0.040000,0.03,2
3,Alief,0.000000,0.015625,0.031250,0.015625,0.031250,0.00,0.000000,0.265625,0.000000,0.000000,0.00,4
4,Astrodome Area,0.021277,0.021277,0.021277,0.063830,0.021277,0.00,0.021277,0.000000,0.021277,0.000000,0.00,0


In [515]:
# df1 id kl_df

In [516]:
# to add latitude/longitude for each neighborhood

kl_merged = kl_merged.join(df1.set_index("Neighborhood"), on="Neighborhood")


In [517]:

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(88, 15)


,Neighborhood,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Burger Joint,American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Cluster Labels,Latitude,Longitude
0,Acres Home,0.000000,0.000000,0.000000,0.000000,0.142857,0.00,0.000000,0.000000,0.000000,0.142857,0.00,3,29.870470,-95.43536
1,Addicks / Park Ten,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0,29.814340,-95.61593
2,Afton Oaks,0.050000,0.050000,0.050000,0.060000,0.020000,0.01,0.000000,0.020000,0.010000,0.040000,0.03,2,29.803759,-95.47215
3,Alief,0.000000,0.015625,0.031250,0.015625,0.031250,0.00,0.000000,0.265625,0.000000,0.000000,0.00,4,29.710880,-95.59569
4,Astrodome Area,0.021277,0.021277,0.021277,0.063830,0.021277,0.00,0.021277,0.000000,0.021277,0.000000,0.00,0,29.691500,-95.38869


In [518]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(88, 15)


,Neighborhood,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Burger Joint,American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Cluster Labels,Latitude,Longitude
87,Willowbrook,0.040000,0.000000,0.020000,0.010000,0.010000,0.030000,0.010000,0.010000,0.010000,0.010000,0.010000,0,29.952400,-95.544630
41,IAH Airport,0.033898,0.050847,0.033898,0.067797,0.016949,0.000000,0.033898,0.000000,0.000000,0.016949,0.016949,0,29.987890,-95.349480
55,Meyerland,0.040000,0.030000,0.040000,0.030000,0.000000,0.040000,0.010000,0.000000,0.020000,0.000000,0.000000,0,29.684530,-95.467460
54,Memorial,0.060000,0.010000,0.000000,0.010000,0.000000,0.030000,0.050000,0.000000,0.000000,0.030000,0.020000,0,29.772630,-95.570920
24,Fourth Ward,0.020000,0.020000,0.000000,0.010000,0.010000,0.020000,0.030000,0.020000,0.000000,0.000000,0.030000,0,29.757620,-95.384490
25,Golfcrest,0.047619,0.047619,0.031746,0.015873,0.047619,0.015873,0.000000,0.015873,0.015873,0.015873,0.000000,0,29.691230,-95.298790
26,Greater Eastwood,0.036585,0.048780,0.048780,0.036585,0.036585,0.012195,0.024390,0.012195,0.024390,0.000000,0.012195,0,29.735990,-95.334980
53,Medical Center,0.020000,0.020000,0.000000,0.010000,0.010000,0.040000,0.020000,0.000000,0.010000,0.000000,0.000000,0,29.711790,-95.393150
20,Eldridge,0.034483,0.034483,0.000000,0.034483,0.068966,0.034483,0.000000,0.000000,0.068966,0.000000,0.000000,0,29.674794,-95.623158
52,Meadowbrook,0.040000,0.010000,0.040000,0.000000,0.000000,0.010000,0.010000,0.010000,0.010000,0.010000,0.010000,0,29.878715,-95.629890


In [519]:
# create map
map_clusters = folium.Map(location=[29.691500, -95.623158], zoom_start=11)

In [520]:

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighborhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [521]:
kl_merged.groupby('Cluster Labels').mean()

,Mexican Restaurant,Pizza Place,Fast Food Restaurant,Sandwich Place,Fried Chicken Joint,Burger Joint,American Restaurant,Vietnamese Restaurant,Chinese Restaurant,Seafood Restaurant,Italian Restaurant,Latitude,Longitude
Cluster Labels,,,,,,,,,,,,,
0,0.030222,0.025456,0.016077,0.024366,0.010695,0.016468,0.015643,0.008315,0.015032,0.009439,0.012502,29.759653,-95.437399
1,0.008773,0.049978,0.100218,0.050279,0.053853,0.021471,0.010871,0.001826,0.015379,0.009865,0.000560,29.839513,-95.370243
2,0.097117,0.043596,0.039028,0.037571,0.035934,0.021157,0.013641,0.012542,0.008913,0.014417,0.007123,29.755647,-95.375008
3,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000,29.870470,-95.435360
4,0.000000,0.015625,0.031250,0.015625,0.031250,0.000000,0.000000,0.265625,0.000000,0.000000,0.000000,29.710880,-95.595690
